In [5]:
pip install selenium

     ---------------------------------------- 6.3/6.3 MB 2.5 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [67]:
review_count = item.find("span", {"class": "a-size-base s-underline-text"}).text
review_count

'(733)'

In [73]:
#import the used libraries
import csv 
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_text):
    """take the search term and generate it in all pages in the web"""
    template = "https://www.amazon.com/s?k={}&crid=15DNYWRD1AUTF&sprefix=ultrawide+monitor%2Caps%2C208&ref=nb_sb_noss_1"
    search_term = search_text.replace(' ','+') #replace empty string to +
    url = template.format(search_term) #then put it in the template url variable
    url += '&page{}' #add page query placeholder
    return url

def extract_record(item):
    """extract and return the data from single records"""
    #description and url 
    atag = item.h2.a #html header
    desc  = atag.text.strip() #header without html tag
    url = 'https://www.amazon.com' + atag.get('href')
    
    #fetch the price
    try:
        price_parent = item.find("span", {"class":"a-price"})
        price = price_parent.find("span", {"class": "a-offscreen"}).text
    except AttributeError: 
        return
    
    #fetch the review and count review 
    try:
        review = item.i.text #product rate
        review_count = item.find("span", {"class": "a-size-base s-underline-text"}).text
    except AttributeError:
        review = ''
        review_count = ''
        
    data = (desc, price, review, review_count, url) #all data
    return data


def main(search_term):
    #chrome driver
    driver = webdriver.Chrome()
    records = []
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close()
    
    # save data to csv file (excel sheet)
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)

In [74]:
main("ultrawide monitor")